In [1]:
import sys
import os

# Añadir la ruta a sys.path
sys.path.append(os.path.abspath('../lib/intro_ia/clase2/hanoi_tower'))

import aima
import hanoi_states
import search


1. PEAS del problema

| Agente | Performance | Environment | Actuators | Sensors |
|--------|-------------|-------------|-----------|---------|
|Torre de Hanoi | Utilizar el menor numero de movimientos posibles| Discos y torres  | Mover discos respetando las reglas del juego| Posición de los discos y estado de las torres|

2. Propiedades del Entorno:

a. Totalmente observable: Sabemos exactamente todas las posiciones de los discos y el estado de las torres.
b. Determinístico: Las acciones tienen resultados predecibles.
c. Secuencial: Cada movimiento de un disco afecta el estado del entorno y las opciones disponibles para los movimientos futuros.
d. Estático: No hay cambios de estados mientras que el agente no intervenga.
e. Discreto: Los estados y acciones son finitos y contables.
f. Agente individual: Solo existe un agente realizando acciones en el entorno.

3.
- Estado: Ubicación de los discos en las torres en un momento específico.
- Espacio de estados: Cantidad de posiciones posibles para los discos en las torres.
- Árbol de búsqueda:
- Nodo de búsqueda:
- Objetivo : Configuración en la que todo los discos han sido movidos de la torre inicial a la torre objetivo.
- Acción: Mover un disco de una torre a otra, siempre y cuando éste no se apoye sobre un disco más chico.
- Frontera: Aquella combinación de posiciones que ya fueron tomadas por el agente versus las posiciones que todavia no fueron exploradas.

In [2]:
class Node:
    def __init__(self, state, parent=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return self.heuristic < other.heuristic


In [3]:
def reconstruct_path(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]


In [4]:
def heuristic(state):
    goal_post = state[2]
    return len(goal_post) - sum(1 for i, disk in enumerate(goal_post) if disk == len(goal_post) - i)


In [5]:
def get_neighbors(state):
    neighbors = []
    for i in range(3):
        if state[i]:
            disk = state[i][-1]
            for j in range(3):
                if i != j and (not state[j] or state[j][-1] > disk):
                    new_state = [peg[:] for peg in state]
                    new_state[i].pop()
                    new_state[j].append(disk)
                    neighbors.append((new_state, 1))
    return neighbors


In [9]:
import heapq

def greedy_best_first_search(start_state, goal_state, heuristic_func, get_neighbors):
    open_list = []
    heapq.heappush(open_list, Node(start_state, heuristic=heuristic_func(start_state)))
    closed_list = set()
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.state == goal_state:
            return reconstruct_path(current_node)
        
        closed_list.add(tuple(tuple(peg) for peg in current_node.state))
        
        for neighbor, cost in get_neighbors(current_node.state):
            neighbor_tuple = tuple(tuple(peg) for peg in neighbor)
            if neighbor_tuple in closed_list:
                continue
            
            neighbor_node = Node(neighbor, current_node, current_node.cost + cost, heuristic_func(neighbor))
            heapq.heappush(open_list, neighbor_node)
    
    return None

search_function = greedy_best_first_search


In [ ]:
import heapq

def a_star_search(start_state, goal_state, heuristic_func, get_neighbors):
    open_list = []
    heapq.heappush(open_list, Node(start_state, heuristic=heuristic_func(start_state)))
    closed_list = set()
    
    while open_list:
        current_node = heapq.heappop(open_list)
        
        if current_node.state == goal_state:
            return reconstruct_path(current_node)
        
        closed_list.add(tuple(tuple(peg) for peg in current_node.state))
        
        for neighbor, cost in get_neighbors(current_node.state):
            neighbor_tuple = tuple(tuple(peg) for peg in neighbor)
            if neighbor_tuple in closed_list:
                continue
            
            g_cost = current_node.cost + cost
            h_cost = heuristic_func(neighbor)
            f_cost = g_cost + h_cost
            
            neighbor_node = Node(neighbor, current_node, g_cost, f_cost)
            heapq.heappush(open_list, neighbor_node)
    
    return None

search_function = a_star_search

In [10]:
start_state = [[5, 4, 3, 2, 1], [], []]
goal_state = [[], [], [5, 4, 3, 2, 1]]

path = search_function(start_state, goal_state, heuristic, get_neighbors)
for step in path:
    print(step)


[[5, 4, 3, 2, 1], [], []]
[[5, 4, 3, 2], [1], []]
[[5, 4, 3], [1], [2]]
[[5, 4, 3], [], [2, 1]]
[[5, 4], [3], [2, 1]]
[[5, 4], [3, 1], [2]]
[[5, 4, 2], [3, 1], []]
[[5, 4, 2], [3], [1]]
[[5, 4], [3, 2], [1]]
[[5, 4], [3, 2, 1], []]
[[5], [3, 2, 1], [4]]
[[5], [3, 2], [4, 1]]
[[5, 2], [3], [4, 1]]
[[5, 2], [3, 1], [4]]
[[5], [3, 1], [4, 2]]
[[5], [3], [4, 2, 1]]
[[5, 3], [], [4, 2, 1]]
[[5, 3, 1], [], [4, 2]]
[[5, 3, 1], [2], [4]]
[[5, 3], [2, 1], [4]]
[[5], [2, 1], [4, 3]]
[[5], [2], [4, 3, 1]]
[[5, 1], [2], [4, 3]]
[[5, 1], [], [4, 3, 2]]
[[5], [], [4, 3, 2, 1]]
[[], [5], [4, 3, 2, 1]]
[[], [5, 1], [4, 3, 2]]
[[2], [5, 1], [4, 3]]
[[2, 1], [5], [4, 3]]
[[2, 1], [5, 3], [4]]
[[2], [5, 3], [4, 1]]
[[], [5, 3, 2], [4, 1]]
[[], [5, 3, 2, 1], [4]]
[[4], [5, 3, 2, 1], []]
[[4, 1], [5, 3, 2], []]
[[4, 1], [5, 3], [2]]
[[4], [5, 3], [2, 1]]
[[4, 3], [5], [2, 1]]
[[4, 3], [5, 1], [2]]
[[4, 3, 2], [5, 1], []]
[[4, 3, 2, 1], [5], []]
[[4, 3, 2, 1], [], [5]]
[[4, 3, 2], [], [5, 1]]
[[4, 3], [2], 